### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [2]:
%pip install grpcio grpcio-tools protobuf
import os
import sys
import time
import grpc
import json
from concurrent import futures
from grpc_tools import protoc

# Step 1: Write the proto file content to disk
proto_content = """
syntax = "proto3";

package validation;

service Validator {
  rpc ValidateRecord(RecordRequest) returns (ValidationResponse);
}

message RecordRequest {
  string record_id = 1;
  string data = 2;  // JSON string representing the record
}

message ValidationResponse {
  bool is_valid = 1;
  repeated string errors = 2;
}
"""

proto_filename = "validation_service.proto"

with open(proto_filename, "w") as f:
    f.write(proto_content)

# Step 2: Compile the proto file programmatically
protoc_args = [
    "",
    f"-I.",
    f"--python_out=.",
    f"--grpc_python_out=.",
    proto_filename,
]

if protoc.main(protoc_args) != 0:
    print("Error: Proto compilation failed")
    sys.exit(1)

# Step 3: Import generated modules
import validation_service_pb2
import validation_service_pb2_grpc

# Step 4: Implement the Validator service
class ValidatorServicer(validation_service_pb2_grpc.ValidatorServicer):
    def ValidateRecord(self, request, context):
        errors = []
        try:
            record = json.loads(request.data)
        except json.JSONDecodeError:
            return validation_service_pb2.ValidationResponse(
                is_valid=False,
                errors=["Invalid JSON format"]
            )
        
        if not request.record_id.strip():
            errors.append("record_id is empty")
        
        if 'amount' not in record:
            errors.append("Missing 'amount' field")
        else:
            try:
                amount = float(record['amount'])
                if amount <= 0:
                    errors.append("'amount' must be greater than zero")
            except ValueError:
                errors.append("'amount' must be a number")
        
        if 'account_id' not in record or not record['account_id']:
            errors.append("Missing or empty 'account_id' field")
        
        is_valid = len(errors) == 0

        return validation_service_pb2.ValidationResponse(
            is_valid=is_valid,
            errors=errors
        )

# Step 5: Start the gRPC server in background thread
def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    validation_service_pb2_grpc.add_ValidatorServicer_to_server(ValidatorServicer(), server)
    server.add_insecure_port("[::]:50051")
    server.start()
    print("Server started on port 50051...")
    return server

# Step 6: Create a client function to test validation
def run_client():
    channel = grpc.insecure_channel("localhost:50051")
    stub = validation_service_pb2_grpc.ValidatorStub(channel)

    valid_record = json.dumps({
        "amount": 150.0,
        "account_id": "A123"
    })
    invalid_record = json.dumps({
        "amount": -10,
        "account_id": ""
    })

    print("Testing valid record...")
    response = stub.ValidateRecord(validation_service_pb2.RecordRequest(record_id="TXN001", data=valid_record))
    print("Valid record validation:", "Valid" if response.is_valid else "Invalid", response.errors)

    print("\nTesting invalid record...")
    response = stub.ValidateRecord(validation_service_pb2.RecordRequest(record_id="", data=invalid_record))
    print("Invalid record validation:", "Valid" if response.is_valid else "Invalid", response.errors)

if __name__ == "__main__":
    server = serve()
    try:
        # Give server time to start
        time.sleep(1)
        run_client()
    finally:
        print("\nStopping server...")
        server.stop(0)
        # Clean up generated files
        os.remove(proto_filename)
        os.remove("validation_service_pb2.py")
        os.remove("validation_service_pb2_grpc.py")

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 16.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [grpcio-tools] [grpcio-tools]
Note: you may need to restart the kernel to use updated packages.
Server started on port 50051...
Testing valid record...
Valid record validation: Valid []

Testing invalid record...
Invalid record validation: Invalid ['record_id is empty', "'amount' must be greater than zero", "Missing or empty 'account_id' field"]

Stopping server...
